In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import spacy
from optimized_anchor import anchor_text, anchor_base
import pickle
import myUtils
from myUtils import *
from models.utils import *
from models.dataset_loader import *
import datetime
import time
import argparse
import os
import sys
sys.path.append('models')
# when apply torchscript to models sometimes
#torch._C._jit_set_texpr_fuser_enabled(False)

warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'huawei-noah/TinyBERT_General_4L_312D'
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
# can be sentiment/spam/dilemma
dataset_name = 'corona'
ds = get_ds(dataset_name)

In [4]:
anchor_examples, _  = preprocess_examples(ds, 200)

Filter:   0%|          | 0/12542 [00:00<?, ? examples/s]

In [5]:
nlp = spacy.load('en_core_web_sm')
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
myUtils.tokenizer = tokenizer

In [6]:
ignored = get_ignored(anchor_examples)
normal_occurences = get_occurences(anchor_examples)
bg = BestGroup('check', normal_occurences, filter_anchors = False, desired_optimize = False)

## notice!

In [7]:
ignored = []

In [8]:
examples = anchor_examples[1:3]

In [ ]:
!nvidia-smi

In [9]:
optimize = True
anchor_text.AnchorText.set_optimize(optimize)
explainer = anchor_text.AnchorText(nlp, ['positive', 'negative'], use_unk_distribution=False)

In [22]:
from transformers import AutoTokenizer
#model = torch.jit.load(f'models/tinybert/{dataset_name}/traced.pt').to(device)
model = load_model(f'models/tinybert/{dataset_name}/model').to(device)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', use_fast = False)
myUtils.model = model
myUtils.tokenizer = tokenizer

In [ ]:
tokenized = explainer.tg.bert_tokenizer(["i [MASK] to [MASK] party yesterday"], 
          add_special_tokens=True, return_tensors="pt", padding=True)["input_ids"]

In [ ]:
tokenized_seperate = [explainer.tg.bert_tokenizer.encode("i went to a party yesterday", 
          add_special_tokens=True) for _ in range(100)]
tokenized_seperate = [torch.tensor([t], device=device) for t in tokenized_seperate]

In [ ]:
x = np.array([1,2,3,4,5,6])

In [ ]:
tokenizer.decode([  101,   103,  2049,  2219, 13109, 28819,  7999,  2126,  1010,
        2009,  4152,  2000,  2017,  1012,  2074,  2066,  1045, 18259,
        2100,   102])

In [84]:
model = load_model(f'models/tinybert/{dataset_name}/model').to(device).eval()
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', 
                                          use_fast = False, do_lower_case = False)

In [170]:
torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(examples, explainer, predict_sentences, ignored, optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
anchor_base.AnchorBaseBeam.best_group = bg
set_seed()
%prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(examples))))

number 0
number 1
[0.9702970297029703]
[0.9803921568627451]
 
*** Profile printout saved to text file 'check/profile.txt'.


         500475 function calls (500079 primitive calls) in 2.554 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.576    2.576 {built-in method builtins.exec}
        1    0.000    0.000    2.576    2.576 <string>:1(<module>)
        1    0.000    0.000    2.576    2.576 myUtils.py:267(compute_explanations)
        2    0.098    0.049    2.575    1.288 myUtils.py:264(get_exp)
        2    0.000    0.000    2.477    1.239 anchor_text.py:267(explain_instance)
        2    0.001    0.001    2.283    1.142 anchor_base.py:255(anchor_beam)
      158    0.013    0.000    2.270    0.014 anchor_text.py:227(sample_fn)
      154    0.000    0.000    2.260    0.015 anchor_base.py:217(<lambda>)
      154    0.005    0.000    2.260    0.015 anchor_base.py:186(complete_sample_fn)
      158    0.019    0.000    2.002    0.013 anchor_text.py:120(sample)
      226    0.004    0.000    1.478    0.007 anchor_t

In [170]:
torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(examples, explainer, predict_sentences, ignored, optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
anchor_base.AnchorBaseBeam.best_group = bg
set_seed()
%prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(examples))))

number 0
number 1
[0.9702970297029703]
[0.9803921568627451]
 
*** Profile printout saved to text file 'check/profile.txt'.


         500475 function calls (500079 primitive calls) in 2.554 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.576    2.576 {built-in method builtins.exec}
        1    0.000    0.000    2.576    2.576 <string>:1(<module>)
        1    0.000    0.000    2.576    2.576 myUtils.py:267(compute_explanations)
        2    0.098    0.049    2.575    1.288 myUtils.py:264(get_exp)
        2    0.000    0.000    2.477    1.239 anchor_text.py:267(explain_instance)
        2    0.001    0.001    2.283    1.142 anchor_base.py:255(anchor_beam)
      158    0.013    0.000    2.270    0.014 anchor_text.py:227(sample_fn)
      154    0.000    0.000    2.260    0.015 anchor_base.py:217(<lambda>)
      154    0.005    0.000    2.260    0.015 anchor_base.py:186(complete_sample_fn)
      158    0.019    0.000    2.002    0.013 anchor_text.py:120(sample)
      226    0.004    0.000    1.478    0.007 anchor_t

In [ ]:
###### my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored,f"profile.pickle")
#%lprun -s -m modified_anchor.anchor_text -m modified_anchor.anchor_base -m myUtils -T profile.txt  my_utils.compute_explanations(list(range(len(anchor_examples))))

In [ ]:
print(datetime.datetime.now())